In [17]:
from collections import Counter
import configparser #changed from ConfigParser to work with Python 3.5
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI
import pickle 
import re 
from collections import defaultdict
import datetime
import codecs
import pickle

def get_twitter(config_file):
    """ Read the config_file and construct an instance of TwitterAPI.
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = configparser.ConfigParser()
    config.read(config_file)
        
    twitter = TwitterAPI(
                   config['twitter']['consumer_key'],
                   config['twitter']['consumer_secret'],
                   config['twitter']['access_token'],
                   config['twitter']['access_token_secret'])
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [3]:
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request is None:
            print ('Found None object')
            break
        if request.status_code == 401:
            print ('Unauthorized data access. The user does not have access to such information')
            break
#         elif "page does not exist" in request.text:
#             print 'page does not exist'
#             break
        elif request.status_code == 200:
            return request
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 1 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)

In [4]:
def get_followers():
    followers = []
    f = open ('iit_followers.txt','r')
    regexp = re.compile(r'p\d+')
    for line in f:
        #print(str(line))
        if regexp.search(str(line)) is not None:
            continue
        else: 
            if (str(line)).startswith('aV'):
                line = line[2:]
                followers.append(line.replace("\n",""))
    followers.sort()
    return followers

followers_list = get_followers()
print (len(followers_list))

6533


In [37]:
#get first 100 user IDs
userIds = []
startIndex = 0
endIndex = 100
while endIndex < len(followers_list):
    userInfo = twitter.request('users/lookup', {'screen_name': followers_list[startIndex:endIndex]})
    for user in userInfo:
        userIds.append(user['id'])
    startIndex += 100
    endIndex += 100
    print ('processed 100 users...')
print ('done')
print (len(userIds))

processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 100 users...
processed 1

In [35]:
userInfo = twitter.request('users/lookup', {'screen_name': 'hpandya2_pa'})
for user in userInfo:
    userIds.insert(0, user['id'])
    print (user['id'])

2885816063


In [39]:
def get_users():
    print ('in get_users()')
    request = twitter.request('statuses/filter', {'follow':userIds[:5000], 'filter_level':'low'})
    timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

    newfile = 'streaming_user_tweets_' + timestamp + '.bin'
    numTweets = 0
    print ('starting collection!!! :)')
    
    while True:
        try:
            for tweet in request:
                print ("got a tweet")
                try:
                    print (tweet['text'])
                    print ('')
                except:
                    continue
                pickle.dump(tweet, open(newfile, 'ab'))
                numTweets += 1
                print ('dumped tweet to file')
                if numTweets > 5000:
                    numTweets = 1
                    timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
                    newfile = 'streaming_user_tweets_' + timestamp + '.bin' 
        except:
            print ('Got error: (sleeping for 60 seconds...)')
            print (request.text)
            sys.stderr.flush()
            time.sleep(60) 
               
get_users()               

in get_users()
starting collection!!! :)
got a tweet
@YearOfRat @raggedtag This was one of those niggly thoughts that stay for decades. Today I decided to invest in clarifying my q with help.

dumped tweet to file
got a tweet
RT @ArchDaily: Steven Holl's "Copenhagen Gateway" Will Finally Go Ahead (Terrifying Bike Lane Included) https://t.co/UcoIYZP9De https://t.c…

dumped tweet to file
got a tweet
Nov 24 - Open House and Member Meeting: Open to the public!
Doors open at 7 for hanging out; ... https://t.co/P0UDey1W5o #tech #event #fb

dumped tweet to file
got a tweet
Love This Show!  https://t.co/AfmwszlrC7

dumped tweet to file
got a tweet
Muting a whatsapp group for 1 year makes me feel powerful 😎😎😏

dumped tweet to file
got a tweet
RT @SuzanneAkhras: #Canada your awesome! #Vancouver #refugee centre 'first of its kind in the world'  https://t.co/vuvJiusp84

dumped tweet to file
got a tweet
New people joining a company and wanting a car after the first month's salary!
Fucks, don't cont

KeyboardInterrupt: 

In [23]:
def readFileAndLabelTweets(skipFirstNTweets):
    tweetFiles = ['streaming_user_tweets_2015-11-21 01:20:40.bin',
                  'streaming_user_tweets_2015-11-23 00:18:26.bin']
    tweetNumber = 1
    tweets = []
    for file in tweetFiles:
        print ('getting tweets from file', file)
        openedFile = open(file, 'rb')
        while True:   
            try:    
                loadedTweet = pickle.load(openedFile)
                
                if skipFirstNTweets + 1 > tweetNumber:
                    tweetNumber+= 1
                    continue
                
                if 'text' in loadedTweet:
                    thisTweet = loadedTweet['text']
                    #only look at unique tweets during classification
                    if thisTweet in tweets:
                        continue
                    else:
                        tweets.append(thisTweet)

                    print ("%d) %s\n" % (tweetNumber, thisTweet))
                    #classify...
                    label = input('Tweet label (1=good, 2=bad, 3=irrelevant, 4=stop): ')
                    if label == '1':
                        print('good\n')
                    elif label == '2':
                        print('bad\n')
                    elif label == '3':
                        print('irrelevant\n')
                    elif label == '4':
                        print('stopping\n')
                        print('labeled %d tweets, ending with tweet #%d' % ((len(tweets) - 1),(tweetNumber - 1)))
                        return
                    
                    #save tweet text with a label
                    tweetTextWithLabel = {'label': label, 'tweetText': loadedTweet['text']}
                    fileName = 'labelsFor' + file
                    pickle.dump(tweetTextWithLabel, open(fileName, 'ab'))
                    
                    tweetNumber += 1
                else:
                    continue
            except EOFError:
                print ("end of file...exiting")
                break

#allow to resume adding labels from the middle of tweets
skipFirstNTweets = 2
readFileAndLabelTweets(skipFirstNTweets)


getting tweets from file streaming_user_tweets_2015-11-21 01:20:40.bin
3) RT @HarvardBiz: You're more likely to believe something if there's a chart. That's not always a good thing: https://t.co/KvSB9cA7QD https:/…

Tweet label (1=good, 2=bad, 3=irrelevant, 4=stop): 4
stopping

labeled 0 tweets, ending with tweet #2


In [31]:
def readTweetTextWithLabel():
    tweetFiles = ['labelsForstreaming_user_tweets_2015-11-21 01:20:40.bin']
    tweetNumber = 1
    tweets = []
    tweetTextWithLabel = []
    for file in tweetFiles:
        print ('getting tweets with label from file', file)
        openedFile = open(file, 'rb')
        while True:   
            try:    
                loadedTweet = pickle.load(openedFile)
                
                #this will only run if multiple labels given to same tweet
                if 'tweetText' in loadedTweet:
                    thisTweetText = loadedTweet['tweetText']
                    #only look at unique tweets during classification
                    if thisTweetText in tweets:
                        continue
                    else:
                        tweets.append(thisTweetText)
                        tweetTextWithLabel.append(loadedTweet)

                    labelText = ''
                    if loadedTweet['label'] == '1':
                        labelText = '[GOOD]'
                    elif loadedTweet['label'] == '2':
                        labelText = '[BAD]'
                    elif loadedTweet['label'] == '3':
                        labelText = '[IRR]'
                    
                    print ("%d) %-6s %s\n" % (tweetNumber, labelText, thisTweetText))
                    tweetNumber += 1
                else:
                    continue
            except EOFError:
                print ("end of file...exiting")
                break
                
readTweetTextWithLabel()

getting tweets with label from file labelsForstreaming_user_tweets_2015-11-21 01:20:40.bin
1) [IRR]  RT @Piclogy: Waves Of Rocks, #Arizona Canyon | Photography by ©Abdulla Almajed https://t.co/7YFLbfZDyk

2) [BAD]  RT @EconSciTech: Why is the American air force sponsoring zoologists at Oxford University? https://t.co/VtkmssBdAs https://t.co/rvDREBNKaT

3) [BAD]  RT @HarvardBiz: You're more likely to believe something if there's a chart. That's not always a good thing: https://t.co/KvSB9cA7QD https:/…

4) [IRR]  RT @dirfooddrink: RT jlmico This Glowing Rock Wants to Monitor Your #InternetOfThings: https://t.co/iJ6ACg0dqL #BigData #DataScienc… https:…

5) [IRR]  RT @dirfooddrink: RT ClearGrip jlmico: Intel snatches an #InternetOfThings exec from Qualcomm: https://t.co/ngC2wdKBVH #BigData #Da… https:…

6) [GOOD] HVAC systems can cause 40% of commercial buildings' energy use! Glad 2 see these companies #energyefficiency focused https://t.co/O6R4gaStuS

7) [IRR]  #AccountingInterns Compensa